In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Instala o Unsloth e atualiza as outras bibliotecas
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --upgrade transformers trl accelerate peft

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-e00ppzoq/unsloth_838bbaafcc8242ca97f95bce32cf04f2
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-e00ppzoq/unsloth_838bbaafcc8242ca97f95bce32cf04f2
  Resolved https://github.com/unslothai/unsloth.git to commit 229e2ecc67756f36316dfcbea42396f59eef44e0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 4.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of trl to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [ ]:
import torch
from unsloth import FastLanguageModel
from transformers import TextStreamer, TrainingArguments
from datasets import load_dataset
from trl import SFTTrainer
import os
INPUT_FILE = "/content/drive/MyDrive/treino_final_tech_challenge/formatted_amazon_products_50k_sanitized.jsonl"
CHECKPOINT_DIR = "/content/drive/MyDrive/treino_final_tech_challenge/checkpoints_final"

print(f"--- Loading dataset: {INPUT_FILE} ---")
dataset = load_dataset("json", data_files=INPUT_FILE, split="train")
print(f"Dataset loaded with {len(dataset)} itens.")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
--- Loading dataset: /content/drive/MyDrive/treino_final_tech_challenge/formatted_amazon_products_50k_sanitized.jsonl ---


Generating train split: 0 examples [00:00, ? examples/s]

Dataset loaded with 50000 itens.


In [ ]:
max_seq_length = 2048
load_in_4bit = True

dtype = torch.float16

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)


==((====))==  Unsloth 2025.9.7: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = ["Generate a detailed description to the title." for _ in examples["title"]]
    inputs       = examples["title"]
    outputs      = examples["content"]
    texts = []
    for instruction, input_text, output_text in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input_text, output_text) + tokenizer.eos_token
        texts.append(text)
    return { "text": texts, }

formatted_dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

{"title": "Canal Boat 1000 Piece Puzzle", "content": "The 1000 piece Canal Boat-Kim Jacobs has been illustrating professionally for over 30 years. Her art has appeared in books and on a wide array of gifts, toys, fine art prints, greeting cards, calendars and other stationery products. Her products have sold in the millions throughout the world. A devoted following has grown to admire and collect Kim's published work and licensed products. Her Puzzle marks the revival of the cherished Pastime Puzzle brand, featuring the the return of unique puzzle piece shapes (whimsies/figurals). No two puzzle pieces are the same and include interlocking border pieces. The pieces feature a clean snap and fit found only among the highest quality jigsaw puzzles."}

In [ ]:
model = FastLanguageModel.get_peft_model(
    model, r=16, target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=False,
    random_state=3407,
)

Unsloth 2025.9.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
print("\n--- Testing not fine-tuned model ---")

base_model, base_tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

FastLanguageModel.get_peft_model(
    base_model, r=16, target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=False,
    random_state=3407,
)


FastLanguageModel.for_inference(base_model)

prompt_para_teste = alpaca_prompt.format(
    "Generate a detailed description to the title.", # instruction
    "Canal Boat 1000 Piece Puzzle", # input
    "", # output
)

inputs = base_tokenizer([prompt_para_teste], return_tensors="pt").to("cuda")
text_streamer = TextStreamer(base_tokenizer)
_ = base_model.generate(**inputs, streamer=text_streamer, max_new_tokens=256)


--- Testing not fine-tuned model ---
==((====))==  Unsloth 2025.9.7: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a detailed description to the title.

### Input:
Canal Boat 1000 Piece Puzzle

### Response:
**A Comprehensive Guide to the Canal Boat 1000 Piece Puzzle**

Welcome to the Canal Boat 1000 Piece Puzzle, a challenging and engaging jigsaw puzzle designed to test your problem-solving skills and provide hours of ente

**A Comprehensive Guide to the Canal Boat 1000 Piece Puzzle**

Welcome to the Canal Boat 1000 Piece Puzzle, a challenging and engaging jigsaw puzzle designed to test your problem-solving skills and provide hours of entertainment. In this guide, we'll walk you through the process of setting up and solving this magnificent puzzle.

**Box Contents:**

* 1000 individually numbered puzzle pieces
* A large, foldable puzzle board
* A brief instruction manual

**Setting Up the Puzzle:**

1. Begin by unfolding the puzzle board and placing it on a flat surface.
2. Carefully remove the 1000 puzzle pieces from the box and sort them into categories based on color, shape, and texture.
3. Use the provided instruction manual to ensure that you're familiar with the puzzle's unique features and challenges.

**Solving the Puzzle:**

1. Start by finding the edge pieces and building the frame of the puzzle. These pieces are usually marked with distinctive shapes or colors that will help you identify them.
2. Once the frame is complete, begin to build the puzzle by connecting adjacent pieces. Use the provided instructions to guide you through the process.
3. As you work your way through the puzzle, use the sorting

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    packing=True,  # mantém o empacotamento
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        warmup_steps=10,
        num_train_epochs=1,
        # max_steps=300,
        learning_rate=2e-4,
        bf16=False,
        fp16=True,
        logging_steps=10,
        save_strategy="steps",
        save_steps=500,
        optim="adamw_8bit",
        output_dir=CHECKPOINT_DIR,
        report_to="none"
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
print("Start trainning")
trainer.train(resume_from_checkpoint = True)
#trainer.train()

Start trainning


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50,000 | Num Epochs = 1 | Total steps = 12,500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
7510,2.004400
7520,2.134900
7530,1.862500
7540,2.139400
7550,2.053400
7560,1.906100
7570,2.043400
7580,1.967900
7590,1.899900
7600,2.013400


Step,Training Loss
7510,2.004400
7520,2.134900
7530,1.862500
7540,2.139400
7550,2.053400
7560,1.906100
7570,2.043400
7580,1.967900
7590,1.899900
7600,2.013400


TrainOutput(global_step=12500, training_loss=0.7880950306701661, metrics={'train_runtime': 9871.6968, 'train_samples_per_second': 5.065, 'train_steps_per_second': 1.266, 'total_flos': 2.1917177356423987e+17, 'train_loss': 0.7880950306701661, 'epoch': 1.0})

In [ ]:
print("\n--- Testing fine-tuned model ---")
FastLanguageModel.for_inference(model)

prompt_para_teste = alpaca_prompt.format(
    "Generate a detailed description to the title.", # instruction
    "Canal Boat 1000 Piece Puzzle", # input
    "", # output
)

inputs = tokenizer([prompt_para_teste], return_tensors="pt").to("cuda")
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=256)


--- Testing fine-tuned model ---
<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a detailed description to the title.

### Input:
Canal Boat 1000 Piece Puzzle

### Response:
Canal Boat 1000 Piece Puzzle<|eot_id|>


In [ ]:
model.save_pretrained("/content/drive/MyDrive/treino_final_tech_challenge/fiap_tech_challenge_model") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/treino_final_tech_challenge/fiap_tech_challenge_model")


('/content/drive/MyDrive/treino_final_tech_challenge/fiap_tech_challenge_model/tokenizer_config.json',
 '/content/drive/MyDrive/treino_final_tech_challenge/fiap_tech_challenge_model/special_tokens_map.json',
 '/content/drive/MyDrive/treino_final_tech_challenge/fiap_tech_challenge_model/chat_template.jinja',
 '/content/drive/MyDrive/treino_final_tech_challenge/fiap_tech_challenge_model/tokenizer.json')

In [ ]:
model.push_to_hub("jardemr/fiap_tech_challenge_22_09", token = "") # Online saving
tokenizer.push_to_hub("jardemr/fiap_tech_challenge_22_09", token = "") # Online saving

README.md:   0%|          | 0.00/596 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          | 45.8kB / 97.3MB            

Saved model to https://huggingface.co/jardemr/fiap_tech_challenge_22_09


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpcl1eaxaj/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
max_seq_length = 2048 
dtype = None 
load_in_4bit = True 

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit", 
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) 

inputs = tokenizer(
[
    alpaca_prompt.format(
        "generate a detailed description for the title.", # instruction
        "Canal Boat 1000 Piece Puzzle", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

In [ ]:
if True:

    from unsloth import FastLanguageModel


    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "jardemr/fiap_tech_challenge_22_09", 
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
        token = "", 
        device_map="auto", 
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference


inputs = tokenizer(
[
    alpaca_prompt.format(
        "generate a detailed description for the title.", # instruction
        "Canal Boat 1000 Piece Puzzle", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)